# **Official NJFL Application and Database**

---

This application aims to keep a historical record of the fantasy football league currently known as NJFL.
<br><br>
***HOW TO RUN THIS APPLICATION:***
- First, run the **Build** section. This sets up global functions and the application entities.
- Then, run the **Create** section. This creates the application and adds data to the database.
- Now, you can run any of the cells in the **Run** section in any order.

## Build

---

You can collapse this section and run all cells to set up the application and database.

### Imports

In [163]:
# Import Libraries
import pandas as pd
import numpy as np
import decimal
from IPython.display import display

### Classes

#### Application

In [198]:
class Application():
  def __init__(self):
    self.db = Database()
    self.visualizer = Visualizer()

  def displayLeagueManagers(self):
    self.visualizer.createLeagueManagersDisplay(self.db.manager_df)
  def displayLeagueManagers2(self):
    self.visualizer.createLeagueManagersDisplay2(self.db.manager_df, self.db.manager_record_df)

  def displayActiveLeagueManagers(self):
    self.visualizer.createLeagueManagersDisplay(self.db.manager_df.loc[self.db.manager_df['active'] > 0])
  def displayActiveLeagueManagers2(self):
    self.visualizer.createLeagueManagersDisplay2(self.db.manager_df.loc[self.db.manager_df['active'] > 0], self.db.manager_record_df)

  def displayManagerAllTimeRecords(self, sort_key):
    self.visualizer.createManagerRecordDisplay(self.db.manager_df, self.db.manager_record_df, sort_key)

  def displayActiveManagerAllTimeRecords(self, sort_key):
    self.visualizer.createManagerRecordDisplay(self.db.manager_df.loc[self.db.manager_df['active'] > 0], self.db.manager_record_df, sort_key)

  def displayRecordsForManager(self, manager):
    display(self.db.manager_record_df.loc[self.db.manager_record_df['abbreviation'] == manager].style.hide_index())

  def displayTotalChampionshipsByManager(self):
    display(self.visualizer.createTotalChampionshipsDisplay(pd.merge(self.db.manager_df, self.db.top_three_finish_df, on='abbreviation')))

  def displayTopThreeFinishes(self):
    self.visualizer.createTopThreeFinishesDisplay(self.db.manager_df, self.db.top_three_finish_df)

#### Database

In [165]:
class Database():
  def __init__(self):
    self.manager_df = ManagerHelper.createManagerDF()
    self.manager_record_df = RecordHelper.createManagerRecordDF()
    self.game_df = GameHelper.createGameDF()
    self.top_three_finish_df = TopThreeFinishHelper().createTopThreeFinishesDF()
    self.playoff_game_df= PlayoffHelper().createPlayoffGameDF()

  def addManagers(self, managers):
    self.manager_df = ManagerHelper.addManagers(self.manager_df, managers)

  def addActiveYears(self):
    self.manager_df = ManagerHelper.addActiveYears(self.manager_df, self.manager_record_df)

  def addGamesForYearAndWeek(self, year, week, games):
    self.game_df = GameHelper.addGamesForYearAndWeek(self.game_df, year, week, games)

  def addLegacyRecordsForYear(self, year, records):
    self.manager_record_df = RecordHelper.addLegacyRecordsForYear(self.manager_record_df, year, records)

  def addPlayoffGamesForYearAndRound(self, year, game_round, playoff_games):
    self.playoff_game_df = PlayoffHelper.addPlayoffGamesForYearAndRound(self.playoff_game_df, year, game_round, playoff_games)

  def addTopThreeFinishesForYear(self, year, finishes):
    self.top_three_finish_df = TopThreeFinishHelper.addTopThreeFinishesForYear(self.top_three_finish_df, year, finishes)

  def addRecordsFromGames(self):
    self.manager_record_df = RecordHelper.addRecordsFromGames(self.game_df, self.manager_record_df)

##### Manager Helper

In [166]:
class ManagerHelper():
  # Create Manager DataFrame
  def createManagerDF():
    df = pd.DataFrame()
    df['name'] = []
    df['abbreviation'] = []
    df['active'] = []
    df = df.astype({
        'active': int })
    return df

  def addManager(name, abbreviation, active, manager_df):
    dict = {
        'name': name,
        'abbreviation': abbreviation,
        'active': active
    }
    manager_df = manager_df.append(dict, ignore_index = True)
    return manager_df

  def addManagers(manager_df, managers):
    for manager in managers:
      name = manager[0]
      abbreviation = manager[1]
      active = manager[2]
      manager_df = ManagerHelper.addManager(name, abbreviation, active, manager_df)
    return manager_df

  def addActiveYears(manager_df, manager_record_df):
    return_df = pd.DataFrame()
    return_df['abbreviation'] = []
    return_df['active_years'] = []
    manager_list = manager_df['abbreviation']
    for manager in manager_list:
      result_df = manager_record_df.loc[manager_record_df['abbreviation'] == manager]
      active_years = ""
      for index, row in result_df.iterrows():
        year = row['season_start_year']
        active_years += str(year) + ", "
      active_years = active_years[0:-2]
      dict = {
          'abbreviation': manager,
          'active_years': active_years
      }
      return_df = return_df.append(dict, ignore_index=True)
    df = pd.merge(manager_df, return_df, on='abbreviation')
    return df

##### Record Helper

In [167]:
class RecordHelper():
  # Create League Manager Records DataFrame
  def createManagerRecordDF():
    df = pd.DataFrame()
    df['abbreviation'] = []
    df['season_start_year'] = []
    df['wins'] = []
    df['losses'] = []
    df['ties'] = []
    df['pf'] = []
    df['pa'] = []
    df = df.astype({
        'season_start_year': int,
        'wins': int,
        'losses': int,
        'ties': int })
    return df

  def addLegacyRecord(manager_record_df, manager, season_start_year, wins, losses):
    dict = {
        'abbreviation': manager,
        'season_start_year': season_start_year,
        'wins': wins,
        'losses': losses,
        'ties': 0
    }
    manager_record_df = manager_record_df.append(dict, ignore_index = True)
    return manager_record_df

  def addLegacyRecordWithTies(manager_record_df, manager, season_start_year, wins, losses, ties):
    dict = {
        'abbreviation': manager,
        'season_start_year': season_start_year,
        'wins': wins,
        'losses': losses,
        'ties': ties
    }
    manager_record_df = manager_record_df.append(dict, ignore_index = True)
    return manager_record_df

  def addLegacyRecordsForYear(manager_record_df, season_start_year, records):
    for record in records:
      manager = record[0]
      wins = record[1]
      losses = record[2]
      if len(record) == 4:
        ties = record[3]
        manager_record_df = RecordHelper.addLegacyRecordWithTies(manager_record_df, manager, season_start_year, wins, losses, ties)
        continue
      manager_record_df = RecordHelper.addLegacyRecord(manager_record_df, manager, season_start_year, wins, losses)
    return manager_record_df

  def addRecordsFromGames(game_df, manager_record_df):
    calculated_df = pd.DataFrame()
    calculated_df['abbreviation'] = pd.Series(dtype='object')
    calculated_df['season_start_year'] = pd.Series(dtype=int)
    calculated_df['wins'] = pd.Series(dtype=int)
    calculated_df['losses'] = pd.Series(dtype=int)
    calculated_df['ties'] = pd.Series(dtype=int)
    calculated_df['pf'] = pd.Series(dtype=float)
    calculated_df['pa'] = pd.Series(dtype=float)
    years_in_game_df = game_df['season_start_year'].unique()
    for year in years_in_game_df:
      year_game_df = game_df.loc[game_df['season_start_year'] == year]
      manager_list1 = np.array(year_game_df['manager1'].unique())
      manager_list2 = np.array(year_game_df['manager2'].unique())
      manager_list = np.concatenate((manager_list1, manager_list2))
      manager_list = np.unique(manager_list)
      for manager in manager_list:
        manager_games1 = year_game_df.loc[year_game_df['manager1'] == manager]
        manager_games2 = year_game_df.loc[year_game_df['manager2'] == manager]
        df = pd.concat([manager_games1, manager_games2])
        wins = 0
        losses = 0
        ties = 0
        pf = 0
        pa = 0
        for index, row in df.iterrows():
          manager1_score = row['manager1_score']
          manager2_score = row['manager2_score']
          if (row['manager1'] == manager):
            if (manager1_score > manager2_score):
              wins += 1
            elif (manager1_score < manager2_score):
              losses += 1
            else:
              ties = RecordHelper.determine_tie(row, manager)
            pf += manager1_score
            pa += manager2_score
          else:
            if (manager2_score > manager1_score):
              wins += 1
            elif (manager2_score < manager1_score):
              losses += 1
            else:
              ties = RecordHelper.determine_tie(row, manager)
            pf += manager2_score
            pa += manager1_score
        dict = {
            'abbreviation': manager,
            'season_start_year': year,
            'wins': wins,
            'losses': losses,
            'ties': ties,
            'pf': pf,
            'pa': pa
        }
        calculated_df = calculated_df.append(dict, ignore_index = True)
    df = pd.concat([manager_record_df, calculated_df])
    return df

  def determine_tie(row, manager):
    return 0

##### Game Helper

In [168]:
class GameHelper():
  # Create Regular Season Games DataFrame
  def createGameDF():
    df = pd.DataFrame()
    df['manager1'] = []
    df['manager1_score'] = []
    df['manager2'] = []
    df['manager2_score'] = []
    df['season_start_year'] = []
    df['week'] = []
    df = df.astype({
        'season_start_year': int, 
        'week': int })
    return df

  def addGame(game_df, manager1, manager1_score, manager2, manager2_score, season_start_year, week):
    dict = {
        'manager1': manager1, 
        'manager1_score': manager1_score, 
        'manager2': manager2, 
        'manager2_score': manager2_score,
        'season_start_year': season_start_year,
        'week': week
    }
    game_df = game_df.append(dict, ignore_index = True)
    return game_df

  def addGamesForYearAndWeek(game_df, year, week, games):
    for game in games:
      manager1 = game[0]
      manager1_score = game[1]
      manager2 = game[2]
      manager2_score = game[3]
      season_start_year = year
      week = week
      game_df = GameHelper.addGame(game_df, manager1, manager1_score, manager2, manager2_score, season_start_year, week)
    return game_df

##### Top Three Finish Helper

In [169]:
# TODO - make this top four

class TopThreeFinishHelper():
  # Create League Manager Top Three Finishes DataFrame
  def createTopThreeFinishesDF(self):
    df = pd.DataFrame()
    df['abbreviation'] = []
    df['season_start_year'] = []
    df['finishing_position'] = []
    df = df.astype({
        'season_start_year': int,
        'finishing_position': int })
    return df

  def addTopThreeFinish(top_three_finish_df, manager, season_start_year, finishing_position):
    dict = {
        'abbreviation': manager,
        'season_start_year': season_start_year,
        'finishing_position': finishing_position
    }
    top_three_finish_df = top_three_finish_df.append(dict, ignore_index=True)
    return top_three_finish_df

  def addTopThreeFinishesForYear(top_three_finish_df, season_start_year, finishes):
    for finish in finishes:
      manager = finish[0]
      finishing_position = finish[1]
      top_three_finish_df = TopThreeFinishHelper.addTopThreeFinish(top_three_finish_df, manager, season_start_year, finishing_position)
    return top_three_finish_df

##### Playoff Helper

In [170]:
class  PlayoffHelper():
  # Create Playoff Game DataFrame
  def createPlayoffGameDF(self):
    df = pd.DataFrame()
    df['manager1'] = []
    df['manager1_rank'] = []
    df['manager1_score'] = []
    df['manager2'] = []
    df['manager2_rank'] = []
    df['manager2_score'] = []
    df['round'] = []
    df['season_start_year'] = []
    df = df.astype({
        'season_start_year': int, 
        'round': int,
        'manager1_rank': int,
        'manager2_rank': int })
    return df

  def addPlayoffGamesForYearAndRound(playoff_game_df, year, round, playoff_games):
    for game in playoff_games:
      manager1 = game[0]
      manager1_rank = game[1]
      manager1_score = game[2]
      manager2 = game[3]
      manager2_rank = game[4]
      manager2_score = game[5]
      playoff_game_df = PlayoffHelper.addPlayoffGame(playoff_game_df, manager1, manager1_rank, manager1_score, manager2, manager2_rank, manager2_score, round, year)
    return playoff_game_df

  def addPlayoffGame(playoff_game_df, manager1, manager1_rank, manager1_score, manager2, manager2_rank, manager2_score, round, year):
    dict = {
        'manager1': manager1, 
        'manager1_rank': manager1_rank,
        'manager1_score': manager1_score, 
        'manager2': manager2, 
        'manager2_rank': manager2_rank,
        'manager2_score': manager2_score,
        'round': round,
        'season_start_year': year
    }
    playoff_game_df = playoff_game_df.append(dict, ignore_index = True)
    return playoff_game_df

##### Sorting Helper

In [171]:
class sort_by():
  # Define Sort Keys
  name = 'name'
  wins = 'wins'
  losses = 'losses'
  ties = 'ties'
  win_pct = 'win_pct'

#### Visualizer

In [553]:
class Visualizer():
  def createLeagueManagersDisplay(self, df):
    pd.set_option('max_colwidth', None)
    df = df.sort_values(by=['name'])
    df = df[['name', 'active_years']]
    df = df.rename(columns={
        'name':'Name', 
        'active_years': 'Active Years' })
    df = df.style.hide_index()
    display(df)

  def createLeagueManagersDisplay2(self, manager_df, manager_record_df):
    pd.set_option('max_colwidth', None)
    df = manager_df.sort_values(by=['name'])
    df = df[['name', 'active_years']]
    df = df.rename(columns={
        'name':'Name', 
        'active_years': 'Active Years' })
    display_df = pd.DataFrame()
    display_df['name'] = []
    years_in_record_df = manager_record_df['season_start_year'].unique()
    dict = {'name':''}
    for year in years_in_record_df:
      display_df[year] = []
      dict[year] = ''
    managers_list = df['Name']
    for manager in managers_list:
      dict = {}
      dict['name'] = manager
      active_years = df.loc[df['Name'] == manager]['Active Years']
      for year in years_in_record_df:
        if str(year) in str(active_years):
          dict[year] = "🟩🟩🟩"
        else:
          dict[year] = ""
      display_df = display_df.append(dict, ignore_index=True)
    display_df = display_df.style.hide_index()
    display(display_df)

  def createManagerRecordDisplay(self, manager_df, manager_record_df, sort_key):
    pd.set_option('max_colwidth', None)
    df = pd.merge(manager_df, manager_record_df, on='abbreviation')
    df = df.groupby('name', as_index=False).sum()
    df = df[['name', 'wins', 'losses', 'ties']]
    df['win_pct'] = pd.Series(dtype='int')
    for index, row in df.iterrows():
      wins = row['wins']
      losses = row ['losses']
      ties = row['ties']
      total_games = wins + losses + ties
      win_percent = wins / total_games
      c = decimal.Decimal(win_percent * 100)
      df.at[index, 'win_pct'] = float(round(c, 3))
    df = df.sort_values(by=[sort_key], ascending=False)
    df = df.rename(columns={
        'name':'Name', 
        'wins': 'Wins',
        'losses': 'Losses',
        'ties': 'Ties',
        'win_pct': 'Win Pct' })
    df = df.style
    df = df.format(precision=3)
    df = df.hide_index()
    display(df)

  def createTotalChampionshipsDisplay(self, df):
    df = df.loc[df['finishing_position'] == 1]
    df = df.groupby('name', as_index=False).sum()
    df = df.sort_values(by=['finishing_position'], ascending=False)
    df = df[['name', 'finishing_position']]
    df = df.rename(columns={'name':'Name','finishing_position': 'Championships'})
    df = df.style.hide_index()
    return df

  def createTopThreeFinishesDisplay(self, manager_df, top_three_finish_df):
    df = pd.merge(manager_df, top_three_finish_df, on='abbreviation')
    df2 = df[['name', 'season_start_year', 'finishing_position']]
    display_df = pd.DataFrame()
    display_df['name'] = []
    years_in_top_3_finishes = top_three_finish_df['season_start_year'].unique()
    dict = {'name':''}
    for year in years_in_top_3_finishes:
      display_df[year] = []
      dict[year] = ''
    dict['total'] = []
    managers_list = df2['name'].unique()
    for manager in managers_list:
      dict = {}
      dict['name'] = manager
      top_three_finishes = df2.loc[df2['name'] == manager][['season_start_year', 'finishing_position']]
      total_top_three = 0
      for year in years_in_top_3_finishes:
        if (top_three_finishes['season_start_year'].eq(year)).any():
          finishing_position = df[(df['season_start_year'] == year) & (df['name'] == manager)]
          finishing_position = finishing_position['finishing_position'].item()
          if (finishing_position == 3):
            dict[year] = "🥉"
          elif (finishing_position == 2):
            dict[year] = "🥈"
          else:
            dict[year] = "🥇"
          total_top_three += 1
        else:
          dict[year] = ""
        dict['total'] = total_top_three
      display_df = display_df.append(dict, ignore_index=True)
    display_df = display_df.astype({'total': int})
    display_df = display_df.sort_values(by=['total'], ascending=False)
    display_df = display_df.rename(columns={'name':'Name','total': 'Total'})
    display_df = display_df.style.hide_index()
    display(display_df)

## Create

---

You can collapse this section and run all cells to create the application and add data to the database.

### Initialize

In [554]:
# Initilaize Application
app = Application()

### Add Managers

In [555]:
# Add Managers

# Reset Managers DataFrame
app.db.manager_df = ManagerHelper.createManagerDF()

CARR = 'CARR'
JRUM = 'JRUM'
TMAC = 'TMAC'
MIKE = 'MIKE'
GARF = 'GARF'
CLAR = 'CLAR'
BERG = 'BERG'
AIKN = 'AIKN'
MOOD = 'MOOD'
WHIT = 'WHIT'
KYLE = 'KYLE'
SMIT = 'SMIT'
NORT = 'NORT'
PELT = 'PELT'
ALEX = 'ALEX'
GTAT = 'GTAT'
COLN = 'COLN'
SEAN = 'SEAN'
HITS = 'HITS'
ZACH = 'ZACH'
BAIL = 'BAIL'

app.db.addManagers([
    ["Brendan Carr", CARR, 1],
    ["Jordan Rumble", JRUM, 1],
    ["Thomas McGowan", TMAC, 1],
    ["Michel Spoto", MIKE, 1],
    ["Jason Garfinkle", GARF, 1],
    ["Brandon Clary", CLAR, 1],
    ["Jared Birnberg", BERG, 1],
    ["Aiken Carter", AIKN, 1],
    ["Brent Moody", MOOD, 1],
    ["Ethan Whittingham", WHIT, 1],
    ["Kyle Brock", KYLE, 1],
    ["Drake Smith", SMIT, 1],
    ["Andrew Norton", NORT, 0],
    ["Adam Pelter", PELT, 0],
    ["Alex Smith", ALEX, 0],
    ["Grant Tatich", GTAT, 0],
    ["Colin Silvia", COLN, 0],
    ["Sean Sessoms", SEAN, 0],
    ["Chris Hitselberger", HITS, 0],
    ["Zach Towner", ZACH, 0],
    ["Bailey Gardin", BAIL, 0]
])

### Add Records

In [556]:
# Add Legacy Records

# Reset Records DB
app.db.manager_record_df = RecordHelper.createManagerRecordDF()

# 2011
app.db.addLegacyRecordsForYear(2011, [
    [SMIT, 7, 7],
    [BERG, 10, 4],
    [JRUM, 7, 7],
    [MOOD, 5, 9],
    [NORT, 8, 6],
    [WHIT, 7, 7],
    [PELT, 8, 6],
    [ALEX, 4, 10]
])

# 2012
app.db.addLegacyRecordsForYear(2012, [
    [KYLE, 8, 5],
    [JRUM, 6, 7],
    [NORT, 9, 4],
    [GTAT, 6, 7],
    [WHIT, 8, 5],
    [COLN, 7, 6],
    [BERG, 8, 5],
    [PELT, 5, 8],
    [SMIT, 4, 9],
    [MOOD, 4, 9]
])

# 2013
app.db.addLegacyRecordsForYear(2013, [
    [GTAT, 7, 6],
    [WHIT, 6, 7],
    [PELT, 7, 6],
    [SEAN, 6, 7],
    [TMAC, 8, 5],
    [BERG, 9, 4],
    [SMIT, 9, 4],
    [JRUM, 9, 4],
    [KYLE, 5, 8],
    [MOOD, 5, 8],
    [NORT, 5, 8],
    [COLN, 2, 11]
])

# 2014
app.db.addLegacyRecordsForYear(2014, [
    [JRUM, 7, 6],
    [TMAC, 7, 6],
    [BERG, 12, 1],
    [WHIT, 8, 5],
    [GTAT, 7, 6],
    [HITS, 6, 7],
    [SMIT, 8, 5],
    [MOOD, 6, 7],
    [CARR, 3, 10],
    [CLAR, 4, 9],
    [KYLE, 6, 7],
    [MIKE, 4, 9]
])

# 2015
app.db.addLegacyRecordsForYear(2015, [
    [COLN, 7, 6],
    [BERG, 12, 1],
    [WHIT, 7, 6],
    [KYLE, 5, 7, 1],
    [SMIT, 4, 8, 1],
    [TMAC, 5, 8],
    [GTAT, 10, 3],
    [HITS, 7, 6],
    [JRUM, 8, 5],
    [CLAR, 4, 9],
    [ZACH, 5, 8],
    [MOOD, 3, 10]
])

# 2016
app.db.addLegacyRecordsForYear(2016, [
    [HITS, 9, 4],
    [ZACH, 6, 7],
    [CARR, 7, 6],
    [WHIT, 8, 5],
    [CLAR, 9, 4],
    [KYLE, 5, 8],
    [JRUM, 9, 4],
    [BERG, 9, 4],
    [SMIT, 4, 9],
    [TMAC, 4, 9],
    [COLN, 3, 10],
    [MOOD, 5, 8]
])

# 2017
app.db.addLegacyRecordsForYear(2017, [
    [SMIT, 8, 5],
    [TMAC, 9, 4],
    [KYLE, 10, 3],
    [WHIT, 9, 4],
    [HITS, 5, 8],
    [ZACH, 6, 7],
    [CLAR, 7, 6],
    [JRUM, 6, 7],
    [CARR, 4, 9],
    [BERG, 5, 8],
    [COLN, 5, 8],
    [MOOD, 4, 9]
])

# 2018
app.db.addLegacyRecordsForYear(2018, [
    [CARR, 9, 4],
    [SMIT, 9, 4],
    [BERG, 10, 3],
    [ZACH, 9, 4],
    [JRUM, 5, 8],
    [MOOD, 7, 6],
    [TMAC, 6, 7],
    [CLAR, 7, 6],
    [HITS, 4, 9],
    [KYLE, 4, 9],
    [WHIT, 4, 9],
    [COLN, 4, 9]
])

# 2019
app.db.addLegacyRecordsForYear(2019, [
    [GARF, 8, 5],
    [KYLE, 6, 7],
    [WHIT, 11, 2],
    [TMAC, 7, 6],
    [CLAR, 9, 4],
    [SMIT, 7, 6],
    [CARR, 5, 8],
    [MOOD, 10, 3],
    [JRUM, 4, 9],
    [BERG, 1, 12],
    [BAIL, 5, 8],
    [COLN, 5, 8]
])


# 2020
app.db.addLegacyRecordsForYear(2020, [
    [CLAR, 7, 6],
    [JRUM, 9, 4],
    [MOOD, 7, 6],
    [WHIT, 10, 3],
    [BERG, 6, 7],
    [BAIL, 7, 6],
    [AIKN, 7, 6],
    [TMAC, 6, 7],
    [GARF, 3, 10],
    [SMIT, 6, 7],
    [KYLE, 4, 9],
    [CARR, 6, 7]
])

# 2021
app.db.addLegacyRecordsForYear(2021, [
    [TMAC, 11, 3],
    [BERG, 10, 4],
    [CARR, 6, 8],
    [AIKN, 7, 7],
    [GARF, 6, 8],
    [MOOD, 9, 5],
    [KYLE, 6, 8],
    [SMIT, 7, 7],
    [WHIT, 5, 9],
    [BAIL, 6, 8],
    [JRUM, 6, 8],
    [CLAR, 5, 9]
])

### Add Games

In [557]:
# Reset Games DF
app.db.game_df = GameHelper.createGameDF()

#### 2022

In [558]:
# Add Games for 2022 Regular Season

# Week 1
app.db.addGamesForYearAndWeek(2022, 1, [
    [CARR, 77.25, WHIT, 129.9],
    [MIKE, 120.65, AIKN, 115.35],
    [KYLE, 130.25, BERG, 95.2],
    [MOOD, 79.75, GARF, 152.9],
    [CLAR, 108.6, SMIT, 68.4],
    [TMAC, 129.15, JRUM, 110.9]
])

# Week 2
app.db.addGamesForYearAndWeek(2022, 2, [
    [CARR, 116.9, TMAC, 140.8],
    [KYLE, 114.85, MIKE, 128.75],
    [AIKN, 139.35, MOOD, 117.2],
    [BERG, 136.95, WHIT, 91.45],
    [GARF, 101.75, CLAR, 103.3],
    [SMIT, 124.65, JRUM, 89.7]
])

# Week 3
app.db.addGamesForYearAndWeek(2022, 3, [
    [CARR, 111.3, SMIT, 76.5],
    [MIKE, 131.9, MOOD, 76.35],
    [WHIT, 100.75, KYLE, 115.4],
    [CLAR, 118.05, AIKN, 115.1],
    [TMAC, 115.4, BERG, 82.2],
    [JRUM, 76.05, GARF, 84.0]
])

# Week 4
app.db.addGamesForYearAndWeek(2022, 4, [
    [GARF, 134.55, CARR, 124.15],
    [WHIT, 101.55, MIKE, 121.15],
    [MOOD, 117.9, CLAR, 132.3],
    [KYLE, 75.6, TMAC, 136.4],
    [AIKN, 84.65, JRUM, 83.2],
    [BERG, 137.55, SMIT, 89.4]
])

# Week 5
app.db.addGamesForYearAndWeek(2022, 5, [
    [CARR, 104.4, AIKN, 130.2],
    [MIKE, 144.4, CLAR, 102.15],
    [TMAC, 81.65, WHIT, 119.65],
    [JRUM, 115.4, MOOD, 103.1],
    [SMIT, 62.0, KYLE, 97.25],
    [GARF, 144.5, BERG, 69.7]
])

# Week 6
app.db.addGamesForYearAndWeek(2022, 6, [
    [MOOD, 96.65, CARR, 84.15],
    [TMAC, 122.0, MIKE, 89.8],
    [CLAR, 103.65, JRUM, 93.85],
    [WHIT, 119.9, SMIT, 67.3],
    [KYLE, 95.65, GARF, 122.7],
    [AIKN, 128.05, BERG, 82.75]
])

# Week 7
app.db.addGamesForYearAndWeek(2022, 7, [
    [CARR, 105.9, CLAR, 88.95],
    [MIKE, 90.0, JRUM, 57.6],
    [SMIT, 86.45, TMAC, 116.7],
    [GARF, 121.65, WHIT, 122.45],
    [BERG, 120.05, MOOD, 122.3],
    [AIKN, 90.75, KYLE, 123.95]
])

# Week 8
app.db.addGamesForYearAndWeek(2022, 8, [
    [JRUM, 98.05, CARR, 125.7],
    [SMIT, 137.9, MIKE, 156.4],
    [TMAC, 146.3, GARF, 73.35],
    [CLAR, 120.95, BERG, 112.0],
    [WHIT, 112.6, AIKN, 130.2],
    [MOOD, 124.1, KYLE, 110.45]
])

# Week 9
app.db.addGamesForYearAndWeek(2022, 9, [
    [MIKE, 111.05, CARR, 92.45],
    [GARF, 104.9, SMIT, 110.05],
    [BERG, 130.25, JRUM, 83.6],
    [AIKN, 72.05, TMAC, 138.35],
    [KYLE, 113.95, CLAR, 114.35],
    [MOOD, 102.85, WHIT, 102.5]
])

# Week 10
app.db.addGamesForYearAndWeek(2022, 10, [
    [CARR, 96.15, BERG, 115.95],
    [GARF, 147.65, MIKE, 119.7],
    [SMIT, 95.65, AIKN, 94.1],
    [JRUM, 86.2, KYLE, 89.25],
    [TMAC, 112.1, MOOD, 105.45],
    [CLAR, 112.05, WHIT, 74.4]
])

# Week 11
app.db.addGamesForYearAndWeek(2022, 11, [
    [KYLE, 110.6, CARR, 115.65],
    [MIKE, 105.5, BERG, 112.4],
    [AIKN, 92.05, GARF, 114.65],
    [MOOD, 124.65, SMIT, 96.7],
    [WHIT, 127.95, JRUM, 85.25],
    [CLAR, 89.95, TMAC, 95.55]
])

# Week 12
app.db.addGamesForYearAndWeek(2022, 12, [
    [CARR, 122.1, WHIT, 82.0],
    [AIKN, 100.95, MIKE, 126.25],
    [BERG, 123.15, KYLE, 99.95],
    [GARF, 129.6, MOOD, 118.8],
    [SMIT, 92.55, CLAR, 70.2],
    [JRUM, 102.2, TMAC, 120.05]
])

# Week 13
app.db.addGamesForYearAndWeek(2022, 13, [
    [TMAC, 122.85, CARR, 98.25],
    [MIKE, 78.25, KYLE, 127.1],
    [MOOD, 108.65, AIKN, 111.15],
    [WHIT, 109.4, BERG, 136.1],
    [CLAR, 109.6, GARF, 99.05],
    [JRUM, 117.45, SMIT, 85.35]
])

# Week 14
app.db.addGamesForYearAndWeek(2022, 14, [
    [CARR, 146.15, SMIT, 89.05],
    [MOOD, 149.7, MIKE, 55.65],
    [KYLE, 90.65, WHIT, 102.55],
    [AIKN, 92.45, CLAR, 75.1],
    [BERG, 104.1, TMAC, 125.9],
    [GARF, 86.2, JRUM, 73.75]
])

In [559]:
# Add Games for 2022 Playoffs

# Round 1
app.db.addPlayoffGamesForYearAndRound(2022, 1, [
    [CARR, 8, 129.4, TMAC, 1, 155.6],
    [BERG, 5, 79.0, CLAR, 4, 103.3],
    [AIKN, 6, 113.5, MIKE, 3, 109.75],
    [MOOD, 7, 75.7, GARF, 2, 113.3]
])

In [560]:
# app.db.addGamesForYearAndWeek(2023, 1, [
#     [CARR, 1, SMIT, 2]
# ])

In [561]:
app.db.addRecordsFromGames()

### Add Active Years

In [562]:
app.db.addActiveYears()

### Add Top Three Finishes

In [563]:
app.db.top_three_finish_df = TopThreeFinishHelper().createTopThreeFinishesDF()

app.db.addTopThreeFinishesForYear(2011,[
    [SMIT, 1],
    [BERG, 2],
    [JRUM, 3]
])

app.db.addTopThreeFinishesForYear(2012, [
    [KYLE, 1],
    [JRUM, 2],
    [NORT, 3]
])

app.db.addTopThreeFinishesForYear(2013, [
    [GTAT, 1],
    [WHIT, 2],
    [PELT, 3]
])

app.db.addTopThreeFinishesForYear(2014, [
    [JRUM, 1],
    [TMAC, 2],
    [BERG, 3]
])

app.db.addTopThreeFinishesForYear(2015, [
    [COLN, 1],
    [BERG, 2],
    [WHIT, 3]
])

app.db.addTopThreeFinishesForYear(2016, [
    [HITS, 1],
    [ZACH, 2],
    [CARR, 3]
])

app.db.addTopThreeFinishesForYear(2017, [
    [SMIT, 1],
    [TMAC, 2],
    [KYLE, 3]
])

app.db.addTopThreeFinishesForYear(2018, [
    [CARR, 1],
    [SMIT, 2],
    [BERG, 3]
])

app.db.addTopThreeFinishesForYear(2019, [
    [GARF, 1],
    [KYLE, 2],
    [WHIT, 3]
])

app.db.addTopThreeFinishesForYear(2020, [
    [CLAR, 1],
    [JRUM, 2],
    [MOOD, 3]
])

app.db.addTopThreeFinishesForYear(2021, [
    [TMAC, 1],
    [BERG, 2],
    [CARR, 3]
])

## Run

---

Each of the following sections will display some table, chart, or graph to visualize NJFL data. Please see the descriptions for each section and run individual cells to see the output.

### League Managers (2011 - 2022)

Shows a table with all the managers who have participated in the NJFL since it's inception.

In [392]:
# app.displayLeagueManagers()
app.displayLeagueManagers2()

name,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Adam Pelter,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,,,,,,,,,
Aiken Carter,,,,,,,,,,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Alex Smith,🟩🟩🟩,,,,,,,,,,,
Andrew Norton,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,,,,,,,,,
Bailey Gardin,,,,,,,,,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,
Brandon Clary,,,,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Brendan Carr,,,,🟩🟩🟩,,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Brent Moody,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Chris Hitselberger,,,,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,,,,
Colin Silvia,,🟩🟩🟩,🟩🟩🟩,,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,,,


### Active League Managers (2022)

Shows a table with all currently active managers in the NJFL.

In [393]:
# app.displayActiveLeagueManagers()
app.displayActiveLeagueManagers2()

name,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Aiken Carter,,,,,,,,,,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Brandon Clary,,,,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Brendan Carr,,,,🟩🟩🟩,,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Brent Moody,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Drake Smith,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Ethan Whittingham,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Jared Birnberg,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Jason Garfinkle,,,,,,,,,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Jordan Rumble,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩
Kyle Brock,,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩,🟩🟩🟩


### League Managers by Year

Shows a table with the managers who participated in any given year

In [394]:
# TODO - use records table to get list of managers by year

### League Manager Records (2011 - 2021)

Shows a table of all league managers records since the inception of the NJFL. Sort values using one of the following values:
- `name`
- `wins`
- `losses` 
- `ties`
- `win_pct`

In [395]:
app.displayManagerAllTimeRecords(sort_by.win_pct)

Name,Wins,Losses,Ties,Win Pct
Jared Birnberg,99,60,0,62.264
Grant Tatich,30,22,0,57.692
Thomas McGowan,76,56,0,57.576
Ethan Whittingham,89,70,0,55.975
Andrew Norton,22,18,0,55.000
Brandon Clary,61,58,0,51.261
Aiken Carter,21,20,0,51.220
Adam Pelter,20,20,0,50.000
Zach Towner,26,26,0,50.000
Jordan Rumble,78,81,0,49.057


### Active League Manager Records (2011-2021)

Shows a table of active league managers records since the inception of the NJFL. Sort values using one of the following values:
- `name`
- `wins`
- `losses` 
- `ties`
- `win_pct`

In [396]:
app.displayActiveManagerAllTimeRecords(sort_by.win_pct)

Name,Wins,Losses,Ties,Win Pct
Jared Birnberg,99,60,0,62.264
Thomas McGowan,76,56,0,57.576
Ethan Whittingham,89,70,0,55.975
Brandon Clary,61,58,0,51.261
Aiken Carter,21,20,0,51.220
Jordan Rumble,78,81,0,49.057
Drake Smith,77,81,1,48.428
Jason Garfinkle,26,28,0,48.148
Michel Spoto,13,14,0,48.148
Kyle Brock,65,79,1,44.828


### Yearly Record by Manager (2011-2021)

Shows a table of a single managers records for every year they participated in the NJFL.

Possible manager abbreviation values can be found in the Create > Add Managers section of this notebook.

In [397]:
manager = CARR

app.displayRecordsForManager(manager)

# TODO - use name instead of abbreviation?, rename columns

abbreviation,season_start_year,wins,losses,ties,pf,pa
CARR,2014,3,10,0,nan,nan
CARR,2016,7,6,0,nan,nan
CARR,2017,4,9,0,nan,nan
CARR,2018,9,4,0,nan,nan
CARR,2019,5,8,0,nan,nan
CARR,2020,6,7,0,nan,nan
CARR,2021,6,8,0,nan,nan
CARR,2022,6,8,0,1520.500000,1527.100000


### League Manager Record by Year

### Total Championships (2011-2021)

Shows a table of total championships for league managers since the inception of the NJFL.

In [565]:
app.displayTotalChampionshipsByManager()

Name,Championships
Drake Smith,2
Brandon Clary,1
Brendan Carr,1
Chris Hitselberger,1
Colin Silvia,1
Grant Tatich,1
Jason Garfinkle,1
Jordan Rumble,1
Kyle Brock,1
Thomas McGowan,1


### Top Three Finishes (2011-2021)

Shows a table of top three finishes for league managers since the inception of the NJFL.

In [564]:
app.displayTopThreeFinishes()

Name,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total
Jared Birnberg,🥈,,,🥉,🥈,,,🥉,,,🥈,5
Jordan Rumble,🥉,🥈,,🥇,,,,,,🥈,,4
Brendan Carr,,,,,,🥉,,🥇,,,🥉,3
Thomas McGowan,,,,🥈,,,🥈,,,,🥇,3
Ethan Whittingham,,,🥈,,🥉,,,,🥉,,,3
Kyle Brock,,🥇,,,,,🥉,,🥈,,,3
Drake Smith,🥇,,,,,,🥇,🥈,,,,3
Jason Garfinkle,,,,,,,,,🥇,,,1
Brandon Clary,,,,,,,,,,🥇,,1
Brent Moody,,,,,,,,,,🥉,,1


### Last Place Finishes (2022)

Shows a table of league last place finishes since this application was created.

In [400]:
# TODO - use games table to calculate regular season record and PF/PA to determine last place finisher

### League Manager PF & PA (2022)

Shows a table of league managers all-time "points for" and "points against" since this application was created.

In [401]:
# TODO - use games table to calculate all-time PF and PA, display total PA/PF, and average PF/PA

### League Manager PF & PA by Year

In [402]:
# TODO - modify above function to handle parameter for year

### Average Game Point Differential

In [403]:
# TODO

### Average PA & PF

In [404]:
# TODO

### Highest/Lowest Points

In [405]:
# TODO

## Scratch Pad

In [515]:
app.displayTopThreeFinishes()

name,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,total
Adam Pelter,,,🥉,,,,,,,,,1
Andrew Norton,,🥉,,,,,,,,,,1
Brandon Clary,,,,,,,,,,🥇,,1
Brendan Carr,,,,,,🥉,,🥇,,,🥉,3
Brent Moody,,,,,,,,,,🥉,,1
Chris Hitselberger,,,,,,🥇,,,,,,1
Colin Silvia,,,,,🥇,,,,,,,1
Drake Smith,🥇,,,,,,🥇,🥈,,,,3
Ethan Whittingham,,,🥈,,🥉,,,,🥉,,,3
Grant Tatich,,,🥇,,,,,,,,,1
